## Modelica Sim Steps

see Intro to Modelica/ Fritzson

1. Constant variables are collected into paremter vector $p$. All other constants can be replaced by their values.
2. Variables declared as input on root model are put in the input vector $u$.
3. Variables whose derivatives appear in the model are put in the state vector $x$.
4. All other variables are collected into a vector y of algebraic variables (their derivatives do not appear in the model).

In [94]:
%matplotlib inline
import casadi as ca
import copy
from pymola.backends.casadi.generator import generate
from pymola.parser import parse
from pymola.backends.casadi.model import Model
from casadi_utils import latex_display, casadi_graph

## Model

In [95]:
model_txt = """
type Voltage = Real(unit="V");
type Current = Real(unit="A");

connector Pin
    Real v;
    flow Real i;
end Pin;

model TwoPin
    Pin p, n;
    Real v;
    Real i;
equation
    v = p.v - n.v;
    0 = p.i + n.i;
    i = p.i;
end TwoPin;

model Resistor
    extends TwoPin;
    parameter Real R;
equation
    v = i*R;
end Resistor;

model Capacitor
    extends TwoPin;
    parameter Real C;
equation
    i = C * der(v);
end Capacitor;

model Inductor
    extends TwoPin;
    parameter Real L;
equation
    v = L * der(i);
end Inductor;

model VsourceAC
    extends TwoPin;
    parameter Real VA = 220;
    parameter Real f = 50;
    constant Real PI=3.14159;
equation
    v = VA*sin(2*PI*f*time);
end VsourceAC;

model Ground
    Pin p;
equation
    p.v = 0;
end Ground;

model SimpleCircuit
    input Real u;
    output Real y;
    Resistor R1(R=10);
    Capacitor C(C=0.01);
    Resistor R2(R=100);
    Inductor L(L=0.1);
    VsourceAC AC;
    Ground G;
equation
    u = L.i;
    y = C.v;
    connect(AC.p, R1.p); // 1. Capacitor circuit
    connect(R1.n, C.p);  //   Wire 2
    connect(C.n, AC.n);  //   Wire 3
    connect(R1.p, R2.p); // 4. Inductor circuit
    connect(R2.n, L.p);  //   Wire 5
    connect(L.n, C.n);   //   Wire 6
    connect(AC.n, G.p);  // 7. Ground
end SimpleCircuit;
"""

## Functions

In [96]:
def sort(eqs, v):
    perm = ca.jacobian(eqs, v).sparsity().btf()
    print(perm)
    row_perm = perm[1]
    col_perm = perm[2]

    # sort equations
    new_eqs = []
    for i in row_perm:
        new_eqs.append(eqs[i])
    new_eqs = ca.vertcat(*new_eqs)

    # sort variables
    new_v = []
    for i in col_perm:
        new_v.append(v[i])
    new_v = ca.vertcat(*new_v)
    return new_eqs, new_v

def split_dae(self):
    x_symbols = set([x.symbol for x in self.states])
    dx_symbols = set([x.symbol for x in self.der_states])
    dae_eqs = []
    alg_eqs = []
    for eq in model.equations:
        if not set(ca.symvar(eq)).isdisjoint(dx_symbols):
            dae_eqs.append(eq)
        else:
            alg_eqs.append(eq)
    dae_eqs = ca.vertcat(*dae_eqs)
    #model_vars = ca.vertcat(*[x.symbol for x in (model.states + model.inputs + model.outputs + model.alg_states + model.der_states)])
    #model_eqs = ca.vertcat(*model.equations)
    #[eq_sort, v_sort] = sort(model_eqs, model_vars)
    return dae_eqs, alg_eqs

def blt_solve(x_symbols, dx_symbols, dae_eqs):
    x = ca.vertcat(*x_symbols)
    dx = ca.vertcat(*dx_symbols)
    perm = ca.jacobian(dae_eqs, dx).sparsity().btf()
    n_blocks = perm[0]
    row_perm = perm[1]
    col_perm = perm[2]
    row_block = perm[3]
    col_block = perm[4]

    # sort equations
    new_dae_eqs = []
    for i in row_perm:
        new_dae_eqs.append(dae_eqs[i])
    dae_eqs = ca.vertcat(*new_dae_eqs)

    # sort variables
    new_x = []
    new_dx = []
    for i in col_perm:
        new_x.append(x[i])
        new_dx.append(dx[i])
    x = ca.vertcat(*new_x)
    dx = ca.vertcat(*new_dx)

    J = ca.jacobian(dae_eqs, dx)

    new_ode = []

    # loop over blocks
    for i_b in range(n_blocks):
        # get variables in the block
        x_b = x[col_block[i_b]:col_block[i_b + 1]]
        dx_b = dx[col_block[i_b]:col_block[i_b + 1]]

        # get equations in the block
        eqs_b = dae_eqs[row_block[i_b]:row_block[i_b + 1]]

        # get the local Jacobian
        J_b = J[row_block[i_b]:row_block[i_b + 1], col_block[i_b]:col_block[i_b+1]]

        J_b.sparsity().spy()

        # if Jb depends on xb, then the state derivative does not enter linearly
        # in the ODE and we cannot solve for the state derivative
        if ca.depends_on(J_b, dx_b):
            raise RuntimeError("Cannot find an explicit epxression for variables {:s}".format(x_b))

        # divide fb into a part which depends on vb and a part which doesn't according to
        # "eqs_b == mul(Jb, vb) + eqs_b_res"
        eqs_b_res = ca.substitute(eqs_b, dx_b, ca.MX(dx_b.shape[0]))

        # solve for vb
        eqs_b_exp = ca.vertsplit(ca.solve(J_b, -eqs_b_res))
        new_ode.append(eqs_b_exp)

    #eliminate inter-dependencies
    #ca.substitute_inplace(dx, ca.vertcat(*new_ode), ca.MX(), False)

## Casadi Generator

In [97]:
from pymola.tree import TreeListener, TreeWalker, flatten
import pymola.ast as ast
import casadi as ca
import numpy as np

class CasadiListener(TreeListener):

    def __init__(self, sym_class=ca.MX, print_depth: int = -1):
        super().__init__()
        self._expr = {}  # type: Dict[ast.Node, Union[ca.MX, ca.SX]]
        self._vars = {}
        self.sym_class = sym_class  # type: Class
        self._time = self.sym('time')
        self._depth = 0
        self._print_depth = print_depth
        self.model = Dae()
        self._built_ins = {
            'sin': 'sin',
            'cos': 'cos',
            'tan': 'tan',
            'sqrt': 'sqrt'
        }

    def sym(self, *args, **kwargs):
        name = args[0]
        if name in self._vars.keys():
            return self._vars[name]
        v = getattr(self.sym_class, 'sym')(*args, **kwargs)
        self._vars[name] = v
        return v

    def get_sym(self, name):
        if name == 'time':
            return self._time
        else:
            return self._vars[name]

    def der(self, s):
        name = s.name()
        der_name = 'der({:s})'.format(name)
        s_dot = self.sym(der_name, s.shape)
        return s_dot

    def const(self, *args, **kwargs):
        return self.sym_class(*args, **kwargs)
        
    def get(self, tree):
        """
        Get the expression representation of the tree.
        """
        if tree not in self._expr.keys():
            raise RuntimeError("no model for node of type", type(tree))
        return self._expr[tree]
    
    def set(self, tree: ast.Node, value):
        """
        Set the expression representation of the tree.
        """
        self._expr[tree] = value

    def enterEvery(self, tree: ast.Node):
        if self._depth <= self._print_depth:
            print('--'*self._depth, tree.__class__.__name__)
        self._depth += 1

    def exitEvery(self, tree: ast.Node):
        self._depth -= 1

    def exitTree(self, tree: ast.Tree):
        assert len(tree.classes) == 1
        name = list(tree.classes.keys())[0]
        cls = tree.classes[name]
        self.set(tree, self.get(cls))
        equations = self.get(tree)
        dae_eq = []
        alg_eq = []
        for eq in equations:
            if ca.depends_on(eq, ca.vertcat(*self.model.der_state)):
                dae_eq.append(eq)
            else:
                alg_eq.append(eq)
        self.model.dae_eq = dae_eq
        self.model.alg_eq = alg_eq

    def exitClass(self, tree: ast.Class):
        eqs = [self.get(eq) for eq in tree.equations]
        self.set(tree, eqs)

    def exitEquationSection(self, tree: ast.EquationSection):
        pass
        
    def exitEquation(self, tree: ast.Equation):
        self.set(tree, self.get(tree.left) - self.get(tree.right))

    def exitExpression(self, tree: ast.Expression):
        if len(tree.operands) == 1:
            right = self.get(tree.operands[0])
            assert isinstance(right, self.sym_class)
            if tree.operator == '+':
                self.set(tree, a)
            elif tree.operator == '-':
                self.set(tree, -right)
            elif tree.operator == 'der':
                self.set(tree, self.der(right))
            elif tree.operator.name in self._built_ins.keys():
                self.set(tree, getattr(ca, self._built_ins[tree.operator.name])(right))
        elif len(tree.operands) == 2:
            left = self.get(tree.operands[0])
            right = self.get(tree.operands[1])
            assert isinstance(left, self.sym_class)
            assert isinstance(right, self.sym_class)
            bin_ops = {
                '+': '__add__',
                '-': '__sub__',
                '*': '__rmul__',
                '/': '__rdiv__',
            }
            if tree.operator in bin_ops.keys():
                self.set(tree, getattr(left, bin_ops[tree.operator])(right))
        
        if tree not in self._expr.keys():
            print('skip expression, #ops: ', len(tree.operands), 'operand:', tree.operator, 'type:', type(tree.operands[0]))

    def exitSymbol(self, tree: ast.Symbol):
        s = self.sym(tree.name)
        type_prefixes = ['parameter', 'constant', 'state', 'input', 'output']
        type_prefix_list = set(type_prefixes).intersection(tree.prefixes)
        if len(type_prefix_list) == 1:
            type_prefix = list(type_prefix_list)[0]
        elif len(type_prefix_list) == 0:
            type_prefix = "alg_state"
        else:
            raise RuntimeError("ambiguous prefxies", tree.prefixes)

        if (type_prefix == "input" or type_prefix == "output") and self._depth > 2:
            type_prefix = "alg_state"

        if tree.name not in getattr(self.model, type_prefix):
            getattr(self.model, type_prefix).append(s)
            if type_prefix == "state":
                s_dot = self.der(s)
                self.model.der_state.append(s_dot)

        self.model.property[s] = {
            'comment': tree.comment,
            'tree.prefixes': tree.prefixes,
            'start': tree.start.value,
            'value': tree.value.value,
            'min': tree.min.value,
            'max': tree.max.value,
            'nominal': tree.nominal.value,
            'dimentaions': [d.value for d in tree.dimensions]
        }
            
    def exitComponentRef(self, tree: ast.ComponentRef):
        #TODO while is Real an issue?
        if tree.name in self._built_ins.keys() or tree.name == "Real":
            return
        try:
            self.set(tree, self.get_sym(tree.name))
        except KeyError as e:
            print('key error', e)

    def exitPrimary(self, tree: ast.Primary):
        try:
            self.set(tree, self.const(tree.value))
        except:
            self.set(tree, tree.value)

In [116]:
class Dae:
    
    def __init__(self):
        self.eq = []
        self.alg_eq = []
        self.state = []
        self.der_state = []
        self.input = []
        self.output = []
        self.alg_state = []
        self.constant = []
        self.parameter = []
        self.property = {}

    def __init__(self):
        self.eq = []
        self.alg_eq = []
        self.state = []
        self.der_state = []
        self.input = []
        self.output = []
        self.alg_state = []
        self.constant = []
        self.parameter = []
        self.property = {}

    def variables(self):
        return self.state + self.alg_state + self.parameter + self.constant + self.input + self.output

    def equations(self):
        return self.dae_eq + self.alg_eq

    def jacobian(self, expr, v):
        return ca.jacobian(ca.vertcat(*getattr(self, expr)),ca.vertcat(*getattr(self, v)))

    def __str__(self):
        s = ""
        for f in ['input', 'output', 'parameter', 'constant', 'state', 'dae_eq', 'alg_state', 'alg_eq']:
            s += '{:s}: {:s}\n'.format(f, str(getattr(self, f)))
        return s

    def evaluate(self, type_name):
        # substitute constants
        v = getattr(self, type_name)
        vals = [self.property[c]['value'] for c in v]
        self.dae_eq = ca.substitute(self.dae_eq, v, vals)
        self.alg_eq = ca.substitute(self.alg_eq, v, vals)
        setattr(self, type_name, [])

    __repr__ = __str__


In [117]:
ref = ast.ComponentRef()
ref.name = "SimpleCircuit"
model_ast = flatten(parse(model_txt), ref)
walker = TreeWalker()
listener = CasadiListener(ca.SX, print_depth=-1)
walker.walk(listener, model_ast)
model = listener.model
model

input: [SX(u)]
output: [SX(y)]
parameter: [SX(R1.R), SX(C.C), SX(R2.R), SX(L.L), SX(AC.VA), SX(AC.f)]
constant: [SX(AC.PI)]
state: [SX(C.v), SX(L.i)]
dae_eq: [SX((C.i-(der(C.v)*C.C))), SX((L.v-(der(L.i)*L.L)))]
alg_state: [SX(R1.p.v), SX(R1.p.i), SX(R1.n.v), SX(R1.n.i), SX(R1.v), SX(R1.i), SX(C.p.v), SX(C.p.i), SX(C.n.v), SX(C.n.i), SX(C.i), SX(R2.p.v), SX(R2.p.i), SX(R2.n.v), SX(R2.n.i), SX(R2.v), SX(R2.i), SX(L.p.v), SX(L.p.i), SX(L.n.v), SX(L.n.i), SX(L.v), SX(AC.p.v), SX(AC.p.i), SX(AC.n.v), SX(AC.n.i), SX(AC.v), SX(AC.i), SX(G.p.v), SX(G.p.i)]
alg_eq: [SX((R1.v-(R1.p.v-R1.n.v))), SX((-(R1.p.i+R1.n.i))), SX((R1.i-R1.p.i)), SX((R1.v-(R1.R*R1.i))), SX((C.v-(C.p.v-C.n.v))), SX((-(C.p.i+C.n.i))), SX((C.i-C.p.i)), SX((R2.v-(R2.p.v-R2.n.v))), SX((-(R2.p.i+R2.n.i))), SX((R2.i-R2.p.i)), SX((R2.v-(R2.R*R2.i))), SX((L.v-(L.p.v-L.n.v))), SX((-(L.p.i+L.n.i))), SX((L.i-L.p.i)), SX((AC.v-(AC.p.v-AC.n.v))), SX((-(AC.p.i+AC.n.i))), SX((AC.i-AC.p.i)), SX((AC.v-(sin((time*(AC.f*(2*AC.PI))))*AC.VA)))

In [127]:
def find_kernel(model, max_iter=100):
    kernel = set()
    eqs = model.equations()
    v = model.variables()
    J = ca.jacobian(ca.vertcat(*model.dae_eq), ca.vertcat(*v)).sparsity()
    count = 0
    while True:
        count += 1
        col_ind = J.get_col()
        cv = ca.vertcat(*[v[i] for i in col_ind])
        col = [v[i] for i in col_ind]
        if set(col).issubset(kernel):
            break
        if count >= max_iter:
            raise RuntimeError("failed to find the kernel in {:d} iterations".format(count))
        for c in col:
            kernel.add(c)
        print(kernel)
        J = ca.jacobian(ca.vertcat(*model.dae_eq), cv).sparsity()
    print("kernel size {:d}, {:d} iterations".format(len(kernel), count))
    return kernel

find_kernel(model)

{SX(C.i), SX(L.L), SX(C.C), SX(L.v)}
{SX(L.L), SX(L.i), SX(R1.p.v), SX(C.i), SX(C.v), SX(C.C), SX(L.v), SX(R1.p.i)}
kernel size 8, 3 iterations


{SX(C.C), SX(C.i), SX(C.v), SX(L.L), SX(L.i), SX(L.v), SX(R1.p.i), SX(R1.p.v)}